# Encoder-Decoder Models for Text Summarization

In [1]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'CS221n/Project/'
FOLDERNAME = 'CS221n/Project/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


## 1. Setup

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras

%matplotlib inline

## 2. Import Dataset

In [3]:
# Gigaword
giga_train, giga_train_info = tfds.load(name="gigaword", split="train", with_info=True)
giga_test, giga_test_info = tfds.load(name="gigaword", split="test")

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

Shuffling and writing examples to /root/tensorflow_datasets/gigaword/1.2.0.incompleteU5WA79/gigaword-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/gigaword/1.2.0.incompleteU5WA79/gigaword-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/gigaword/1.2.0.incompleteU5WA79/gigaword-test.tfrecord


Dataset gigaword downloaded and prepared to /root/tensorflow_datasets/gigaword/1.2.0. Subsequent calls will reuse this data.


Gigaword Examples

In [31]:
print(tf.__version__)
# tfds.as_dataframe(ds=giga_train, ds_info=None)
print(giga_train)

2.3.0
<DatasetV1Adapter shapes: {document: (), summary: ()}, types: {document: tf.string, summary: tf.string}>


## 3. Naive Encoder-Decoder model
One-shot Model: https://machinelearningmastery.com/encoder-decoder-models-text-summarization-keras/

In [12]:
vocab_size = 100
src_txt_length = 100
sum_txt_length = 10

# encoder input model
inputs = tf.keras.Input(shape=(src_txt_length, ))
encoder1 = tf.keras.layers.Embedding(vocab_size, 128)(inputs)
encoder2 = tf.keras.layers.LSTM(128)(encoder1)
encoder3 = tf.keras.layers.RepeatVector(sum_txt_length)(encoder2)

# decoder output model
decoder1 = tf.keras.layers.LSTM(128, return_sequences=True)(encoder3)
outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size, activation='softmax'))(decoder1)

# tie it together
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')